In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import xarray
import firedrake
from firedrake import max_value
import icepack
from icepack.constants import ice_density as ρ_I, water_density as ρ_W

Read in the mesh.

In [ ]:
mesh = firedrake.Mesh("DenmanThermalMesh.msh")

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
firedrake.triplot(mesh, axes=ax)
ax.legend(loc="upper right");

Read in the velocity data.

In [ ]:
measures_filename = icepack.datasets.fetch_measures_antarctica()
measures = xarray.open_dataset(measures_filename)

In [ ]:
V = firedrake.VectorFunctionSpace(mesh, "CG", 1)
u_obs = icepack.interpolate((measures["VX"], measures["VY"]), V)

There are some missing points, so just mask them out with 0.
**TODO**: Fix this awfulness.

In [ ]:
u_obs.dat.data[np.isnan(u_obs.dat.data_ro)] = 0

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(u_obs, axes=ax)
fig.colorbar(colors);

Read in the bed, thickness, and surface elevation data.

In [ ]:
bedmachine_filename = icepack.datasets.fetch_bedmachine_antarctica()
bedmachine = xarray.open_dataset(bedmachine_filename)

In [ ]:
Q = firedrake.FunctionSpace(mesh, "CG", 1)

b = icepack.interpolate(bedmachine["bed"], Q)
h_obs = icepack.interpolate(bedmachine["thickness"], Q)
h_min = firedrake.Constant(10.0)
h = firedrake.Function(Q).interpolate(max_value(h_min, h_obs))
s = firedrake.Function(Q).interpolate(max_value(b + h, (1 - ρ_I / ρ_W) * h))

fields_2d = {"bed": b, "thickness": h, "surface": s, "velocity": u_obs}

Plot where the ice appears to be floating or not.

In [ ]:
δh = firedrake.Function(Q).interpolate(s - (1 - ρ_I / ρ_W) * h)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.set_xlim((2.4e6, 2.8e6))
ax.set_ylim((-550e3, -200e3))
colors = firedrake.tripcolor(δh, vmin=0.0, vmax=100.0, axes=ax)
fig.colorbar(colors);

Fetch the ALBMAP data set for other variables (see [here](https://doi.pangaea.de/10.1594/PANGAEA.734145)).
The last method call below replaces the coordinate names in the original data set (`"x1"`, `"y1"`) with the coordinate names that the icepack interpolation routines are expecting.
The variable `ghffm` refers to a geothermal heat flux estimate from Fox Maule and others (2005), and `ghfsr` an estimate from Shapiro and Ritzwoller (2004).
We also need the surface temperature and accumulation.

In [ ]:
!wget --no-clobber https://store.pangaea.de/Publications/LeBrocq_et_al_2010/ALBMAPv1.nc.zip
!unzip -n ALBMAPv1.nc.zip

In [ ]:
albmap = xarray.open_dataset("ALBMAPv1.nc").rename({"x1": "x", "y1": "y"})
[var for var in albmap.data_vars]

In [ ]:
T = icepack.interpolate(albmap["temp"], Q)
a = icepack.interpolate(albmap["acca"], Q)

fields_2d.update({"temperature": T, "accumulation": a})

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True)
for ax in axes:
    ax.set_aspect("equal")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

colors = firedrake.tripcolor(T, axes=axes[0])
fig.colorbar(colors, ax=axes[0], orientation="horizontal", pad=0.025, label="surface temperature (C)")
colors = firedrake.tripcolor(a, axes=axes[1])
fig.colorbar(colors, ax=axes[1], orientation="horizontal", pad=0.025, label="accumulation (m / yr)");

Estimate the melt rate, assuming that most of the glacier is roughly in steady state, by applying a smoother kernel to the quantity
$$\dot m \approx \dot a - \nabla\cdot hu.$$
We additionally constrain the result to be positive.

In [ ]:
from firedrake import inner, grad, div, dx
m = firedrake.Function(Q)
m_target = a - div(h * u_obs)
α = firedrake.Constant(10e3)
J = 0.5 * (inner(m - m_target, m - m_target) + α**2 * inner(grad(m), grad(m))) * dx
F = firedrake.derivative(J, m)
problem = firedrake.NonlinearVariationalProblem(F, m)
params = {"solver_parameters": {"snes_type": "vinewtonrsls"}}
solver = firedrake.NonlinearVariationalSolver(problem, **params)
lower = firedrake.Function(Q)
upper = firedrake.Function(Q)
upper.assign(1e3)
solver.solve(bounds=(lower, upper))
fields_2d.update({"melt": m})

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(m, vmax=+5, axes=ax)
fig.colorbar(colors);

In [ ]:
num_layers = 1
raw_mesh3d = firedrake.ExtrudedMesh(mesh, num_layers)
Q3D = firedrake.FunctionSpace(raw_mesh3d, "CG", 1, vfamily="R", vdegree=0)
z_b = firedrake.Function(Q3D)
z_s = firedrake.Function(Q3D)
z_b.dat.data[:] = fields_2d["bed"].dat.data_ro[:]
z_s.dat.data[:] = fields_2d["surface"].dat.data_ro[:]
Vc = raw_mesh3d.coordinates.function_space()
x, y, ζ = firedrake.SpatialCoordinate(raw_mesh3d)
X = firedrake.Function(Vc).interpolate(firedrake.as_vector((x, y, z_b + ζ * z_s)))
mesh3d = firedrake.Mesh(X)

In [ ]:
V3D = firedrake.VectorFunctionSpace(mesh3d, "CG", 1, vfamily="R", vdegree=0, dim=2)
u2d = firedrake.Function(V3D)
u2d.dat.data[:] = fields_2d["velocity"].dat.data_ro[:]

In [ ]:
Q3D = firedrake.FunctionSpace(mesh3d, "CG", 1, vfamily="R", vdegree=0)
fields_3d = {}
for key in ["melt", "accumulation", "bed", "thickness", "surface"]:
    field = firedrake.Function(Q3D)
    field.dat.data[:] = fields_2d[key].dat.data_ro[:]
    fields_3d[key] = field

x, y, z = firedrake.SpatialCoordinate(mesh3d)

b = fields_3d["bed"]
m = fields_3d["melt"]
s = fields_3d["surface"]
a = fields_3d["accumulation"]

δh = s - (1 - ρ_I / ρ_W) * h

W = firedrake.FunctionSpace(mesh3d, "DG", 1, vfamily="CG", vdegree=1)
w_0 = u2d[0] * b.dx(0) + u2d[1] * b.dx(1) - m
w = firedrake.Function(W).project(w_0 - (u2d[0].dx(0) + u2d[1].dx(1)) * (z - b))

In [ ]:
u = firedrake.as_vector((u2d[0], u2d[1], w))